In [1]:
import import_ipynb
import numpy as np
import pandas as pd
import datetime as dt
import os
import math
import my_method as mm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, date, time, timedelta
from pickle import dump

from my_method import find_overtime, fill_missing_index, df_interpolate, sep_from5to10, make_log_percip, check_date, make_past_percip, make_include_past

importing Jupyter notebook from my_method.ipynb


In [2]:
# 파일 경로
data_path = ['ASOS/']

# 파일 목록 불러오기
file_list = []
for row in range(1):
    file_list.append(os.listdir(data_path[row]))

# df 리스트
ASOS = []

# df 리스트를 만드는 함수
def make_df_list(data_path, file_list, label):
    df_list = []
    col_list = []
    # label값에 따라 원하는 열 선택
    col_list.append(['일시', '기온(°C)', '풍속(m/s)', '습도(%)', '증기압(hPa)',
                     '이슬점온도(°C)', '현지기압(hPa)', '전운량(10분위)', '중하층운량(10분위)', #'운형(운형약어)',
                     '지면온도(°C)', '강수량(mm)'])
    col_list.append(['일시', '기온(°C)', '풍향(deg)', '풍속(m/s)', '강수량(mm)'])
    col_list.append(['일시', '기온(°C)', '풍향(deg)', '풍속(m/s)', '습도(%)', '강수량(mm)'])

    for i in file_list:
        file_path = os.path.join(data_path, i)
        df = pd.DataFrame()
        df = pd.read_csv(file_path, usecols = col_list[label], parse_dates = ['일시'], encoding = 'CP949')
        df_list.append(df)

    return df_list

ASOS = make_df_list(data_path[0], file_list[0], 0)

# 강수량 결측 제거
for row in range(22):
    ASOS[row]['강수량(mm)'] = ASOS[row]['강수량(mm)'].fillna(0)
    
#display(ASOS[21])

In [3]:
# 결측치 선형 보간

ASOS_itpl = df_interpolate(ASOS)

for row in range(22):
    print('ASOS_{} 결측치\n'.format(row + 0), ASOS_itpl[row].isnull().sum().sort_values, '\n--------------------')

ASOS_0 결측치
 <bound method Series.sort_values of 일시             0
기온(°C)         0
강수량(mm)        0
풍속(m/s)        0
습도(%)          0
증기압(hPa)       0
이슬점온도(°C)      0
현지기압(hPa)      0
전운량(10분위)      0
중하층운량(10분위)    0
지면온도(°C)       3
dtype: int64> 
--------------------
ASOS_1 결측치
 <bound method Series.sort_values of 일시             0
기온(°C)         0
강수량(mm)        0
풍속(m/s)        0
습도(%)          0
증기압(hPa)       0
이슬점온도(°C)      0
현지기압(hPa)      0
전운량(10분위)      0
중하층운량(10분위)    0
지면온도(°C)       3
dtype: int64> 
--------------------
ASOS_2 결측치
 <bound method Series.sort_values of 일시             0
기온(°C)         0
강수량(mm)        0
풍속(m/s)        0
습도(%)          0
증기압(hPa)       0
이슬점온도(°C)      0
현지기압(hPa)      0
전운량(10분위)      0
중하층운량(10분위)    0
지면온도(°C)       3
dtype: int64> 
--------------------
ASOS_3 결측치
 <bound method Series.sort_values of 일시             0
기온(°C)         0
강수량(mm)        0
풍속(m/s)        0
습도(%)          0
증기압(hPa)       0
이슬점온도(°C)      0
현지기압(hPa)      0
전운량

In [4]:
# 데이터 구간 분리

ASOS_sep = sep_from5to10(ASOS_itpl)

# 결측치 제거 확인

pd.set_option('display.max_rows', 40)
for row in range(22):
    print('ASOS_{} 결측치\n'.format(row + 0), ASOS_sep[row].isnull().sum().sort_values, '\n--------------------')

# 열 이름 변경

for row in range(22):
    ASOS_sep[row].columns = ['일시', '기온', '강수량', '풍속', '습도',
                                '증기압', '이슬점', '현지기압', '전운량', '중하층', '지면온도']

#display(ASOS_sep[0])

ASOS_0 결측치
 <bound method Series.sort_values of 일시             0
기온(°C)         0
강수량(mm)        0
풍속(m/s)        0
습도(%)          0
증기압(hPa)       0
이슬점온도(°C)      0
현지기압(hPa)      0
전운량(10분위)      0
중하층운량(10분위)    0
지면온도(°C)       0
dtype: int64> 
--------------------
ASOS_1 결측치
 <bound method Series.sort_values of 일시             0
기온(°C)         0
강수량(mm)        0
풍속(m/s)        0
습도(%)          0
증기압(hPa)       0
이슬점온도(°C)      0
현지기압(hPa)      0
전운량(10분위)      0
중하층운량(10분위)    0
지면온도(°C)       0
dtype: int64> 
--------------------
ASOS_2 결측치
 <bound method Series.sort_values of 일시             0
기온(°C)         0
강수량(mm)        0
풍속(m/s)        0
습도(%)          0
증기압(hPa)       0
이슬점온도(°C)      0
현지기압(hPa)      0
전운량(10분위)      0
중하층운량(10분위)    0
지면온도(°C)       0
dtype: int64> 
--------------------
ASOS_3 결측치
 <bound method Series.sort_values of 일시             0
기온(°C)         0
강수량(mm)        0
풍속(m/s)        0
습도(%)          0
증기압(hPa)       0
이슬점온도(°C)      0
현지기압(hPa)      0
전운량

In [5]:
# 강수량 + 1에 ln
log_ASOS = make_log_percip(ASOS_sep)

In [6]:
# 01 ~ 19년은 train set, 20 ~ 22년은 test set
# train은 pd, 전부 합친 train set

train = pd.DataFrame()

for i in range(19):
    train = pd.concat([train, log_ASOS[i]], axis = 0, ignore_index = True)

test1 = log_ASOS[19]
test2 = log_ASOS[20]
test3 = log_ASOS[21]

In [7]:
train.drop(['일시'], axis = 1, inplace = True)
test1.drop(['일시'], axis = 1, inplace = True)
test2.drop(['일시'], axis = 1, inplace = True)
test3.drop(['일시'], axis = 1, inplace = True)

In [8]:
# n시간전 강수량 포함
train_include_past = make_include_past(train, 6)
test1_include_past = make_include_past(test1, 6)
test2_include_past = make_include_past(test2, 6)
test3_include_past = make_include_past(test3, 6)

In [9]:
display(train_include_past)
display(test1_include_past)
display(test2_include_past)
display(test3_include_past)

,기온,풍속,습도,증기압,이슬점,현지기압,전운량,중하층,지면온도,log_percip,1시간전,2시간전,3시간전,4시간전,5시간전,6시간전
0,17.4,2.4,70.0,13.900000,11.9,1004.1,7.0,4.0,17.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17.6,0.6,67.0,12.633333,10.3,1003.8,7.0,4.0,16.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,17.8,0.5,55.0,11.366667,8.7,1003.5,7.0,4.0,16.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17.1,1.7,52.0,10.100000,7.1,1003.2,7.0,4.0,15.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17.1,1.2,47.0,9.466667,6.1,1003.3,8.0,4.0,15.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83899,17.3,2.4,78.0,15.400000,13.4,1009.3,7.0,7.0,15.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83900,17.3,2.3,78.0,15.400000,13.4,1009.4,6.0,6.0,15.700000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83901,17.0,2.8,80.0,15.500000,13.5,1009.3,7.0,7.0,15.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83902,16.8,1.7,81.0,15.500000,13.5,1009.8,8.0,8.0,14.900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,기온,풍속,습도,증기압,이슬점,현지기압,전운량,중하층,지면온도,log_percip,1시간전,2시간전,3시간전,4시간전,5시간전,6시간전
0,17.8,3.4,79,16.1,14.1,1004.2,9.0,9.0,17.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17.3,2.2,79,15.6,13.6,1004.1,9.0,9.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,16.8,1.7,81,15.5,13.5,1003.7,8.0,8.0,14.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,16.5,1.4,83,15.5,13.5,1003.6,9.0,9.0,14.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16.6,0.6,82,15.5,13.5,1003.5,9.0,9.0,15.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,15.2,3.4,62,10.6,7.9,1011.9,7.0,4.0,12.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4412,15.1,2.8,59,10.1,7.1,1011.9,8.0,8.0,12.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4413,14.9,2.7,58,9.7,6.6,1011.5,10.0,10.0,12.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4414,14.9,2.6,57,9.6,6.4,1011.5,9.0,5.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,기온,풍속,습도,증기압,이슬점,현지기압,전운량,중하층,지면온도,log_percip,1시간전,2시간전,3시간전,4시간전,5시간전,6시간전
0,10.8,0.3,81,10.4,7.6,996.1,9.0,9,11.2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,10.8,2.3,83,10.7,8.0,996.5,10.0,6,11.3,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,10.5,1.8,82,10.4,7.5,995.6,10.0,3,11.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,10.2,1.9,82,10.2,7.2,994.4,10.0,5,10.8,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,10.2,2.0,83,10.3,7.4,994.3,10.0,6,10.6,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,15.7,1.0,66,11.7,9.3,1012.5,8.0,3,13.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4412,15.3,1.3,70,12.1,9.8,1012.7,9.0,8,12.6,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4413,14.3,0.3,77,12.5,10.3,1013.3,7.0,7,12.8,0.530628,0.000000,0.000000,0.0,0.0,0.0,0.0
4414,13.5,0.6,84,12.9,10.8,1013.5,5.0,5,11.8,0.000000,0.530628,0.000000,0.0,0.0,0.0,0.0


,기온,풍속,습도,증기압,이슬점,현지기압,전운량,중하층,지면온도,log_percip,1시간전,2시간전,3시간전,4시간전,5시간전,6시간전
0,10.9,3.0,74,9.6,6.4,1005.9,3,0,8.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.3,2.1,67,8.4,4.4,1005.6,0,0,7.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9.8,2.0,67,8.1,3.9,1005.4,0,0,6.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9.5,2.0,68,8.0,3.8,1004.9,0,0,6.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8.6,1.0,73,8.1,4.0,1004.4,1,0,5.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,15.7,1.1,68,12.1,9.8,1012.5,1,1,13.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4412,14.7,1.9,76,12.7,10.5,1012.2,0,0,12.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4413,14.2,1.7,79,12.8,10.6,1012.4,0,0,11.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4414,13.5,1.8,82,12.6,10.4,1012.6,0,0,11.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
train_include_past.to_csv('make_data/train_ASOS_include_past.csv', encoding = 'CP949')
test1_include_past.to_csv('make_data/test1_ASOS_include_past.csv', encoding = 'CP949')
test2_include_past.to_csv('make_data/test2_ASOS_include_past.csv', encoding = 'CP949')
test3_include_past.to_csv('make_data/test3_ASOS_include_past.csv', encoding = 'CP949')